In [2]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import pickle
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
from spellchecker import SpellChecker
import pyttsx3

In [1]:
class SignLanguageApp:
    def __init__(self):
        self.engine = pyttsx3.init()
        
        self.spell = SpellChecker()
        
        self.model = load_model('hope.keras')
        with open('label_encoder.pkl', 'rb') as f:
            self.label_encoder = pickle.load(f)
        
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.7
        )
        self.mp_drawing = mp.solutions.drawing_utils
        
        self.cap = cv2.VideoCapture(0)
        
        self.current_gesture = ""
        self.current_word = ""
        self.current_sentence = ""
        self.recent_predictions = []
        self.processing_active = False
        self.hand_in_region = False
        
        self.roi_x = 100
        self.roi_y = 100
        self.roi_width = 400
        self.roi_height = 400
        
        self.setup_ui()
        
    def setup_ui(self):
        """Setup the user interface"""
        self.root = tk.Tk()
        self.root.title("Sign Language Detection")
        self.root.geometry("1200x800")
        self.root.configure(bg='gray')
        
        self.video_frame = tk.Frame(self.root, bg='gray')
        self.video_frame.pack(side=tk.LEFT, padx=20, pady=20)
        
        self.control_frame = tk.Frame(self.root, bg='gray', width=400)
        self.control_frame.pack(side=tk.RIGHT, padx=20, pady=20, fill=tk.Y)
        
        self.video_label = tk.Label(self.video_frame, bg='black')
        self.video_label.pack()
        
        tk.Label(self.control_frame, text="Gesture:", font=('Arial', 16, 'bold'), bg='gray', fg='black').pack(anchor='w', pady=(0,5))
        self.gesture_label = tk.Label(self.control_frame, text="", font=('Arial', 16), bg='white', relief="solid", width=30, height=1)
        self.gesture_label.pack(anchor='w', pady=(0,15))
        
        tk.Label(self.control_frame, text="Word:", font=('Arial', 16, 'bold'), bg='gray', fg='black').pack(anchor='w', pady=(0,5))
        self.word_label = tk.Label(self.control_frame, text="", font=('Arial', 16), bg='white', relief="solid", width=30, height=1)
        self.word_label.pack(anchor='w', pady=(0,15))
        
        tk.Label(self.control_frame, text="Suggestions:", font=('Arial', 16, 'bold'), bg='gray', fg='black').pack(anchor='w', pady=(0,5))
        self.suggestions_frame = tk.Frame(self.control_frame, bg='gray')
        self.suggestions_frame.pack(anchor='w', fill=tk.X, pady=(0,15))
        
        tk.Label(self.control_frame, text="Sentence:", font=('Arial', 16, 'bold'), bg='gray', fg='black').pack(anchor='w', pady=(0,5))
        self.sentence_label = tk.Label(self.control_frame, text="", font=('Arial', 16), bg='white', relief="solid", width=30, height=2, wraplength=350)
        self.sentence_label.pack(anchor='w', pady=(0,15))
        
        button_frame = tk.Frame(self.control_frame, bg='gray')
        button_frame.pack(pady=20)
        
        button_style = {'font': ('Arial', 12), 'width': 10, 'height': 2}
        self.start_button = tk.Button(button_frame, text="Start", command=self.toggle_processing, **button_style)
        self.start_button.pack(side=tk.LEFT, padx=5)
        
        tk.Button(button_frame, text="Clear", command=self.clear_all, **button_style).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="Read", command=self.read_sentence, **button_style).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="Exit", command=self.cleanup, **button_style).pack(side=tk.LEFT, padx=5)
        
        self.video_loop()
        
    def toggle_processing(self):
        """Toggle processing on/off"""
        self.processing_active = not self.processing_active
        self.start_button.config(text="Stop" if self.processing_active else "Start")
        
    def read_sentence(self):
        """Read the current sentence using text-to-speech"""
        if self.current_sentence:
            self.engine.say(self.current_sentence)
            self.engine.runAndWait()
            
    def process_landmarks(self, hand_landmarks):
        """Process hand landmarks for prediction"""
        try:
            landmarks = []
            for landmark in hand_landmarks.landmark:
                landmarks.append([landmark.x, landmark.y, landmark.z])
            return np.array(landmarks).reshape(1, 21, 3)
        except Exception as e:
            print(f"Error processing landmarks: {e}")
        return None
    
    def check_hand_in_region(self, hand_landmarks, frame_width, frame_height):
        """Check if hand is in the defined region"""
        for landmark in hand_landmarks.landmark:
            x, y = int(landmark.x * frame_width), int(landmark.y * frame_height)
            if not (self.roi_x < x < self.roi_x + self.roi_width and self.roi_y < y < self.roi_y + self.roi_height):
                return False
        return True
    
    def video_loop(self):
        """Main video processing loop"""
        try:
            ret, frame = self.cap.read()
            if ret:
                frame = cv2.flip(frame, 1)
                
                cv2.rectangle(frame, (self.roi_x, self.roi_y), (self.roi_x + self.roi_width, self.roi_y + self.roi_height), (0, 255, 0), 2)
                
                if self.processing_active:
                    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results = self.hands.process(rgb_frame)
                    
                    if results.multi_hand_landmarks:
                        hand_landmarks = results.multi_hand_landmarks[0]
                        
                        height, width = frame.shape[:2]
                        self.hand_in_region = self.check_hand_in_region(hand_landmarks, width, height)
                        
                        if self.hand_in_region:
                            self.mp_drawing.draw_landmarks(frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)
                            
                            landmarks = self.process_landmarks(hand_landmarks)
                            if landmarks is not None:
                                predictions = self.model.predict(landmarks, verbose=0)
                                predicted_class = np.argmax(predictions, axis=1)
                                predicted_label = self.label_encoder.inverse_transform(predicted_class)[0]
                                
                                if predicted_label.lower() != "open":
                                    confidence = np.max(predictions)
                                    if confidence > 0.5:  
                                        self.recent_predictions.append(predicted_label)
                                        if len(self.recent_predictions) > 5:
                                            self.recent_predictions.pop(0)
                                        
                                        if len(self.recent_predictions) >= 3:
                                            stable_prediction = max(set(self.recent_predictions), key=self.recent_predictions.count)
                                            
                                            if stable_prediction != self.current_gesture:
                                                self.current_gesture = stable_prediction
                                                self.current_word += stable_prediction
                                                self.update_labels()
                        else:
                            cv2.putText(frame, "Please keep hand inside the region", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
                img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                imgtk = ImageTk.PhotoImage(image=img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)
        
        except Exception as e:
            print(f"Error in video loop: {e}")
        
        self.root.after(10, self.video_loop)
    
    def update_labels(self):
        """Update all display labels"""
        self.gesture_label.config(text=self.current_gesture)
        self.word_label.config(text=self.current_word)
        self.sentence_label.config(text=self.current_sentence)
        
        for widget in self.suggestions_frame.winfo_children():
            widget.destroy()
        
        if self.current_word:
            tk.Button(
                self.suggestions_frame,
                text=self.current_word,
                font=('Arial', 12),
                command=lambda: self.use_suggestion(self.current_word)
            ).pack(side=tk.LEFT, padx=5)
        
        suggestions = set()
        for word in self.spell.word_frequency.keys():
            if word.lower().startswith(self.current_word.lower()):
                suggestions.add(word.upper())
        
        for suggestion in list(suggestions)[:3]:
            if suggestion != self.current_word:
                btn = tk.Button(
                    self.suggestions_frame,
                    text=suggestion,
                    font=('Arial', 12),
                    command=lambda s=suggestion: self.use_suggestion(s)
                )
                btn.pack(side=tk.LEFT, padx=5)
    
    def use_suggestion(self, word):
        """Update the sentence with selected suggestion"""
        self.current_word = ""
        self.current_sentence += " " + word
        self.update_labels()
    
    def clear_all(self):
        """Clear all fields and reset variables"""
        self.current_gesture = ""
        self.current_word = ""
        self.current_sentence = ""
        self.recent_predictions.clear()
        self.update_labels()
    
    def cleanup(self):
        """Release resources and close application"""
        self.cap.release()
        cv2.destroyAllWindows()
        self.root.quit()
        
    def run(self):
        """Run the application"""
        self.root.mainloop()

# Run the app
if __name__ == "__main__":
    app = SignLanguageApp()
    app.run()


NameError: name 'pyttsx3' is not defined